### Векторные представления слов (word embeddings)

Урок 3. Embedding word2vec fasttext
Скачиваем датасет

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

Или данные можно скачать тут https://github.com/IlyaGusev/gazeta

Пример работы с ним:

from corus import load_ods_gazeta path = 'gazeta.csv.gz' records = load_ods_gazeta(path) next(records)

что надо сделать:

на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
Проверить насколько хорошо работают подходы

In [1]:
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import json

In [3]:
#import gensim.downloader as api

In [4]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import notebook

In [5]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

In [6]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [7]:
assert True

# Preprocess for models fitting

sentences = []
with open("../data/gazeta_train.json", "r", encoding="utf-8") as fin:
    for line in notebook.tqdm(fin):
        line = json.loads(line)                     
        spls = preprocess_txt(line['text'])
        sentences.append(spls)
        c += 1
        if c > 10000:
            break

0it [00:00, ?it/s]

In [8]:
sentences = [i for i in sentences if len(i) > 2]

In [9]:
sentences[0]

['«по',
 'итог',
 '2011',
 'чистый',
 'отток',
 'составить',
 'примерно',
 '80',
 'млрд',
 'следующий',
 '—',
 '20',
 'млрд',
 'ожидать',
 'полугодие',
 '2012',
 'начаться',
 'приток',
 'капитала»',
 '—',
 'заявить',
 '«интерфаксу»',
 'замминистра',
 'экономический',
 'развитие',
 'андрей',
 'клепач',
 'официальный',
 'прогноз',
 'вывод',
 'капитал',
 'россия',
 'становиться',
 'пессимистичный',
 'лето',
 'власть',
 'полагать',
 'страна',
 'уйти',
 '35',
 'млрд',
 'сентябрь',
 'минэкономразвития',
 'назвать',
 'цифра',
 '50',
 'млрд',
 'начало',
 'ноябрь',
 'центробанк',
 'пересмотреть',
 'оценка',
 '70',
 'млрд',
 'очередной',
 'изменение',
 'прогноз',
 'ожидать',
 'расчёт',
 'центробанк',
 'январь',
 '—',
 'октябрь',
 'чистый',
 'отток',
 'капитал',
 'достигнуть',
 '64',
 'млрд',
 'причём',
 'последний',
 'месяц',
 'ускориться',
 'сентябрь',
 'составить',
 '14',
 'млрд',
 'октябрь',
 '—',
 '13',
 'млрд',
 'среднее',
 'ежемесячный',
 'отток',
 '6—8',
 'млрд',
 'полугодие',
 '«после',


In [10]:
# Word2Vec?

In [11]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)

In [12]:
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

In [13]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

with open("../data/gazeta_train.json", "r", encoding="utf-8") as f:
    for line in notebook.tqdm(f):
        n_w2v = 0
        n_ft = 0
        line = json.loads(line)
        index_map[counter] = line['text']
        question = preprocess_txt(line['text'])
                
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 10000:
            break

w2v_index.build(10)
ft_index.build(10)

0it [00:00, ?it/s]

True

In [14]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5)
    return [index_map[i] for i in answers]

In [22]:
TEXT01 = "выборы в законодательный орган"

In [24]:
get_response(TEXT01, w2v_index, modelW2V, index_map)

['Конституционный суд огласит в понедельник решение по запросу 109 депутатов Госдумы , в котором они просили проверить на соответствие Конституции закон о прямых губернаторских выборах. Справедливороссы и коммунисты, подавшие свою жалобу в КС 13 ноября, уверены, что порядок избрания губернаторов через предложенные властями «муниципальный» и «президентский» фильтры (заявители называют их «цензы»), принятый Госдумой в апреле, нарушает избирательные права граждан. Согласно этому порядку, прописанному весной в законах «Об общих принципах организации законодательных и исполнительных органов власти субъектов РФ» и «Об основных гарантиях избирательных прав», претендент на губернаторский пост должен заручиться поддержкой от 5 до 10% муниципальных депутатов или глав муниципальных образований (конкретный барьер в этом диапазоне регулирует уже региональное законодательство). При этом на губернаторских выборах в Москве или Санкт-Петербурге кандидата должны поддержать не менее чем 75% муниципальных

In [25]:
get_response(TEXT01, ft_index, modelFT, index_map)

['Конституционный суд РФ (КС) в среду приступил к рассмотрению дела о порядке формирования органов местного самоуправления в муниципальных районах. Житель подмосковного Наро-Фоминска считает, что действующее законодательство по этой части нарушает конституционные права граждан избирать и быть избранными. По мнению заявителя, это не что иное, как «массовые политические репрессии» против всех граждан России. Суть вопроса заключается в том, как следует выбирать представительные органы МСУ второго уровня (советы депутатов муниципальных районов). В настоящее время, в соответствии со ст. 35 закона № 131-ФЗ «Об общих принципах организации местного самоуправления в Российской Федерации», советы могут формироваться двумя способами. Первый – традиционные прямые выборы депутатов, как это делается на уровне городских и сельских поселений. Второй же предполагает, что в совет муниципального района входят представители от каждого из поселений (ими могут быть главы поселений или депутаты поселенческих

In [19]:
TEXT02 = "данные рождаемости и смертности в России"

In [20]:
get_response(TEXT02, w2v_index, modelW2V, index_map)

['После событий в Западном Бирюлеве с программными заявлениями относительно будущей миграционной политики выступили и московские, и федеральные власти. Мэр Москвы Сергей Собянин в многочисленных интервью, которые он раздавал в последние дни, как заклинание повторяет, что он против интеграции мигрантов из Средней Азии — то есть тех, кто вызывает наибольший гнев части москвичей и кого наиболее активно «доят» взятками полиция и миграционные службы. «Если кого-то и интегрировать, то все-таки выходцев из стран более подходящего культурного ареала с точки зрения языка, истории...» — считает мэр Москвы. И мэрия, надо признать, делает шаги в этом направлении — вербует рабочую силу на Украине, в Белоруссии, Молдавии. Поскольку никаких принципиальных различий в миграционном законодательстве для жителей Узбекистана с Таджикистаном, с одной стороны, и Украины с Белоруссией, с другой, не существует и они вряд ли возможны, такая политика столичных властей только усилит превращение мигрантов из Средн

In [21]:
get_response(TEXT02, ft_index, modelFT, index_map)

['В среду Федеральная миграционная служба России должна была выдать беглому сотруднику ЦРУ Эдварду Сноудену справку, позволяющую ему покинуть аэропорт Шереметьево. Справку Сноудену должен был передать ставший публичным посредником между ним и российскими властями член Общественной палаты , адвокат Анатолий Кучерена. Однако вопреки прогнозам Кучерена вышел из транзитной зоны Шереметьево один. «Вопрос пока не решен. К сожалению, та ситуация, которая сложилась, нестандартна для России, столкнулись с бюрократией: документы пока находятся на рассмотрении... Подождем некоторое время в надежде на то, что этот вопрос будет разрешен в ближайшие дни», — сказал адвокат. Оказалось, что справка никаких прав передвигаться по России Сноудену не дает. При этом Кучерена заявил, что Сноуден останется в России, так как хочет посмотреть страну. Понятно, что посмотреть страну Сноуден сможет, только получив справку ФМС. Кучерена также сообщил, что Сноуден планирует остаться жить в России на постоянной основ